## To define a trader

In stockAir, data sets, artificial intelligence models are included in one trader, and model learning is performed. You buy and sell stocks with this generated trader.

**First,** create an empty list of traders.

In [ ]:
lst_trader = [] 

**Next,** define the trader.
When defining a trader, you will need the following:

#### 1. `conditional_buyer` 
Object that determines acquisition by dataset filtering conditions

- `Conditional_buyer.condition:` Add a dataset filtering condition to the condition method. (For example, only data with a transaction price (end price x transaction volume) of more than 1 billion won.)


#### 2. `machinelearning_buyer` 
Object that determines acquisition by machine learning model

- `machinelearning_buyer.algorithm:` Adds a user-defined model to the algorithm method. You can add all machine learning models, such as models in the sklearn package or deep learning models that you define yourself.


#### 3. `SubSeller`
Objects that determine the sale of all of the following days


#### 4. `Trader Object Definitions`
- `Trader.name:` Trader Distinguishable Name
- `Trader.label:` Set dependent variable type for trader's dataset (regression: reg, classification: class&0.02)
    - The default value for the dependent variable of stockAI is the rate of change (next_change) at the closing price of the next day.
    - Class&0.02 means binary classification as 1 if the species and rate of change are greater than 0.02 the next day, and 0 if not.

- `trader.buyer:` Put two buyers in the form of buyer ([conditional_buyer, machine learning_buyer]).

- `trader.seller:` Put it in the form of a seller (Subseller).

(그림 설명도 추가하자) 

-------- 

The following example defines a model in a machine learning model package that has already been created.

### ex1) LightGBM 

In [ ]:
from lightgbm import LGBMClassifier

# conditional_buyer: Object that determines acquisition based on data filtering conditions 
b1_lg = sai.ConditionalBuyer()

def sampling1(df): # Create a conditional function
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) # Remove exceptions that exceed upper and lower limits
    condition2 = df.D0_trading_value >= 1000000000 # condition 1: Transaction amount of more than 1 billion won 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) # condition 2: Today's stock price change rate is more than 5%
    condition = condition1 & condition2 & condition3
    return condition

b1_lg.condition = sampling1  # Define the condition function directly (sampling1) and store it in the condition property 


# machinelearning_buyer: Object that determines acquisition by machine learning model
b2_lg = sai.MachinelearningBuyer()

# Save user-defined models to algorithm properties
scale_pos_weight = round(72/28 , 2)
params = {  'random_state' : 42,
            'scale_pos_weight' : scale_pos_weight,
            'learning_rate' : 0.1, 
            'num_iterations' : 1000,
            'max_depth' : 4,
            'n_jobs' : 30,
            'boost_from_average' : False,
            'objective' : 'binary' }

b2_lg.algorithm =  LGBMClassifier( **params )


# SubSeller: Object that determines selling all of the following days
sell_all = sai.SubSeller() 


# Trader Object   
t1 = sai.Trader()
t1.name = 'saiLightGBM' # Trader's name
t1.label = 'class&0.02' # Set the Trader dependent variable (do not set if it is regression analysis) 
t1.buyer = sai.Buyer([b1_lg, b2_lg]) # [ conditional buyer, machinelearning buyer ] 
t1.seller = sai.Seller(sell_all)

lst_trader.append(t1)

### ex2) XGBoost 

In [ ]:
from xgboost import XGBClassifier

b1_xgb = sai.ConditionalBuyer() 

def sampling2(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_xgb.condition = sampling2


b2_xgb = sai.MachinelearningBuyer()  

scale_pos_weight = round(72/28 , 2)
b2_xgb.algorithm = XGBClassifier(random_state = 42,
                   n_jobs=30,
                   scale_pos_weight=scale_pos_weight,
                   learning_rate=0.1,
                   max_depth=4,
                   n_estimators=1000,
                   )  

sell_all = sai.SubSeller()


t2 = sai.Trader()
t2.name = 'saiXGboost' 
t2.label = 'class&0.02' 
t2.buyer = sai.Buyer([b1_xgb, b2_xgb])
t2.seller = sai.Seller(sell_all) 

lst_trader.append(t2) 

### ex3) LogisticRegression 

In [ ]:
from sklearn.linear_model import LogisticRegression

b1_lr = sai.ConditionalBuyer()

def sampling3(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_lr.condition = sampling3


b2_lr = sai.MachinelearningBuyer()  

b2_lr.algorithm = LogisticRegression()


sell_all = sai.SubSeller() 


t3 = sai.Trader()
t3.name = 'saiLogisticRegression'  
t3.label = 'class&0.02' 
t3.buyer = sai.Buyer([b1_lr, b2_lr]) 
t3.seller = sai.Seller(sell_all)

lst_trader.append(t3) 

### ex4) Support Vector Machine

In [ ]:
from sklearn.svm import SVC

b1_sv = sai.ConditionalBuyer()

def sampling4(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_sv.condition = sampling4 


b2_sv = sai.MachinelearningBuyer()  

b2_sv.algorithm = SVC() 


sell_all = sai.SubSeller() 


t4 = sai.Trader()
t4.name = 'saiSupportVectorMachine'  
t4.label = 'class&0.02' 
t4.buyer = sai.Buyer([b1_sv, b2_sv]) 
t4.seller = sai.Seller(sell_all)

lst_trader.append(t4) 

### ex5) Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

b1_dt = sai.ConditionalBuyer()

def sampling5(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_dt.condition = sampling5 


b2_dt = sai.MachinelearningBuyer()  

b2_dt.algorithm = DecisionTreeClassifier() 


sell_all = sai.SubSeller() 


t5 = sai.Trader()
t5.name = 'saiDecisionTree'  
t5.label = 'class&0.02' 
t5.buyer = sai.Buyer([b1_dt, b2_dt]) 
t5.seller = sai.Seller(sell_all)

lst_trader.append(t5) 

### ex6) RandomForest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

b1_rf = sai.ConditionalBuyer()

def sampling6(df):  
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_rf.condition = sampling6 


b2_rf = sai.MachinelearningBuyer()  

b2_rf.algorithm = RandomForestClassifier() 


sell_all = sai.SubSeller() 


t6 = sai.Trader()
t6.name = 'saiDecisionTree'  
t6.label = 'class&0.02' 
t6.buyer = sai.Buyer([b1_rf, b2_rf]) 
t6.seller = sai.Seller(sell_all)

lst_trader.append(t6) 

----------

The following is a deep learning model, which is an example of defining an lstm model using keras. You need to add one more property called `data_transform` to the deep learning model.

### ex7) LSTM 
⭐ if you are using a deep learning model, you must define a function that is defined as a **two-dimensional structure**, and when designing the model, **you must set input_shape to the size of the dataset.**

In [ ]:
from tensorflow import keras

b1_ls = sai.ConditionalBuyer()

def sampling7(df): 
    condition1 = (-0.3 <= df.D0_Change) & (df.D0_Change <= 0.3) 
    condition2 = df.D0_trading_value >= 1000000000 
    condition3 = (-0.05 >= df.D0_Change) | (0.05 <= df.D0_Change) 
    condition = condition1 & condition2 & condition3
    return condition

b1_ls.condition = sampling7 


b2_ls = sai.MachinelearningBuyer()

# ⭐ User-defined functions (users who want deep learning modeling)
def transform(data): # A function that converts into a two-dimensional structure / data: list (lst_time_series)
    data_2d = []
    n_col = int(len(data[0]) / 10) 
    for row in data:      
        data_2d.append([])
        for i in range(0, len(row), n_col):
            data_2d[-1].append(row[i:i+n_col])
    
    return np.array(data_2d)
    

# Directly define a two-dimensional structure transformation function (transform) and store it in the data_transform property
b2_ls.data_transform = transform 

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(10, 48)))
model.add(keras.layers.LSTM(128, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(64, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(32, activation='relu', return_sequences=False))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
    
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(0.05,decay_steps=100000,decay_rate=0.96)), 
    loss="binary_crossentropy",
    metrics=['accuracy'])

b2_ls.algorithm =  model


sell_all = sai.SubSeller() 


t7 = sai.Trader()
t7.name = 'saiLSTM' 
t7.label = 'class&0.02' 
t7.buyer = sai.Buyer([b1_ls, b2_ls]) 
t7.seller = sai.Seller(sell_all)

lst_trader.append(t7)

Define a trader and **append each trader on the last line to lst_trader**, the list containing the first trader you created.

In this way, you can define traders and use `lst_trader` to learn models and trade stocks for multiple traders. The `tutorials/02.different_models.ipynb` file shows an example of it, so it would be good to refer to it.